# Testing wrap_openai + anthropic

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
import anthropic
from langsmith import traceable
from langsmith.wrappers import wrap_anthropic

# Text input

In [3]:
# simple text only example 
client = wrap_anthropic(anthropic.Anthropic())

@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    return "During this morning's meeting, we solved all world conflict."

@traceable(name="Chat Pipeline")
def chat_pipeline(question: str):
    context = my_tool(question)
    messages = [
        { "role": "user", "content": f"Question: {question}\nContext: {context}"}
    ]
    messages = client.messages.create(
      model="claude-sonnet-4-20250514",
      messages=messages,
      max_tokens=1024,
      system="You are a helpful assistant. Please respond to the user's request only based on the given context."
    )
    return messages



chat_pipeline("Can you summarize this morning's meetings?")


Message(id='msg_0152REzNZPjSGyi84ZgGuwdX', content=[TextBlock(citations=None, text="Based on the context provided, this morning's meeting achieved a resolution to all world conflict. However, I should note that the context given appears to be quite brief and potentially unrealistic, so I can only summarize what was explicitly stated: the meeting resulted in solving all world conflict.\n\nIf you're looking for a more detailed summary of actual meetings that took place, you may need to provide more specific and realistic context about what was actually discussed, who attended, and what outcomes were achieved.", type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=53, output_tokens=102, server_tool_use=None, service_tier='standard'))

# PDF inputs base64

In [9]:
import base64
import httpx
from IPython.display import Image, display

pdf_url = "https://pdfobject.com/pdf/sample.pdf"
pdf_data = base64.b64encode(httpx.get(pdf_url).content).decode("utf-8")


response_multimodal = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": [
                {"type": "document", "source": {
                    "type": "base64",
                    "media_type": "application/pdf",
                    "data": pdf_data
                }},
                {"type": "text", "text": "what is in this pdf?"}

        ]}    ],
)


## bug: we are logging <anthropic.Omit object at 0x11084ba10>" 

we should strip this message before we log traces. we also show it in the "Default" rendering in the system message

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=1e823f71-c77b-40ba-a20f-a5806f20ff66&peeked_trace=1e823f71-c77b-40ba-a20f-a5806f20ff66

# Image inputs base64

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

response_multimodal = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": [
                {"type": "image", "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": image_data
                }},
                {"type": "text", "text": "what is in this image?"}

        ]}    ],
)

# PDF inputs URL

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"

response_multimodal = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": [
                {"type": "image", "source": {
                    "type": "url",
                    "url": image_url
                }},
                {"type": "text", "text": "what is in this image?"}

        ]}    ],
)

# Tools

In [ ]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}],
)

## bug: we don't show that the tool was called in the "Tools" section 

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=d190eb5b-9648-4ac6-9569-7008ba1806ea&peeked_trace=d190eb5b-9648-4ac6-9569-7008ba1806ea

# JSON Mode

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image_media_type = "image/jpeg"
image_data = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")

message = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    tools=[
        {
            "name": "record_summary",
            "description": "Record summary of an image using well-structured JSON.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "key_colors": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "r": {
                                    "type": "number",
                                    "description": "red value [0.0, 1.0]",
                                },
                                "g": {
                                    "type": "number",
                                    "description": "green value [0.0, 1.0]",
                                },
                                "b": {
                                    "type": "number",
                                    "description": "blue value [0.0, 1.0]",
                                },
                                "name": {
                                    "type": "string",
                                    "description": "Human-readable color name in snake_case, e.g. \"olive_green\" or \"turquoise\""
                                },
                            },
                            "required": ["r", "g", "b", "name"],
                        },
                        "description": "Key colors in the image. Limit to less than four.",
                    },
                    "description": {
                        "type": "string",
                        "description": "Image description. One to two sentences max.",
                    },
                    "estimated_year": {
                        "type": "integer",
                        "description": "Estimated year that the image was taken, if it is a photo. Only set this if the image appears to be non-fictional. Rough estimates are okay!",
                    },
                },
                "required": ["key_colors", "description"],
            },
        }
    ],
    tool_choice={"type": "tool", "name": "record_summary"},
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_data,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
)

# Web search tool

In [ ]:
client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "What's the weather in NYC?"
        }
    ],
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
)

In [ ]:
# Code execution tool

In [ ]:
client.beta.messages.create(
    model="claude-sonnet-4-5",
    betas=["code-execution-2025-08-25"],
    max_tokens=4096,
    messages=[{
        "role": "user",
        "content": "Calculate the mean and standard deviation of [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
    }],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    }]
)

# Files API

In [ ]:
# Upload the image file
with open("blog.jpg", "rb") as f:
    file_upload = client.beta.files.upload(file=("blog.jpg", f, "image/jpeg"))

# Use the uploaded file in a message
message = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["files-api-2025-04-14"],
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "file",
                        "file_id": file_upload.id
                    }
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ]
        }
    ],
)

## bug: Doesn't render the image 
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=df4fe0cb-d36e-4216-af23-0c694aca3b68&peeked_trace=df4fe0cb-d36e-4216-af23-0c694aca3b68

# MCP 

In [ ]:
client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    messages=[{
        "role": "user",
        "content": "Get the last 5 tickets from linear"
    }],
    mcp_servers=[{
        "type": "url",
        "url": "https://mcp.linear.app/sse",
        "name": "linear-mcp",
        "authorization_token": "XXX",
        "tool_configuration": {
            "enabled": True,
            "allowed_tools": ["list_issues", "get_issue"]
        }
        }],
    betas=["mcp-client-2025-04-04"]
)

## gap: we don't render tools available from an MCP server

If a list of tools is included via allowed_tools, we should render them like we do regular tools (see mcp-rendering.png)

# Citations 

In [8]:
# text/plain citation 
client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "The grass is green. The sky is blue."
                    },
                    "title": "My Document",
                    "context": "This is a trustworthy document.",
                    "citations": {"enabled": True}
                },
                {
                    "type": "text",
                    "text": "What color is the grass and sky?"
                }
            ]
        }
    ]
)

Message(id='msg_01ST6DrKVs6hP2viG2mMDbYG', content=[TextBlock(citations=[CitationCharLocation(cited_text='The grass is green. ', document_index=0, document_title='My Document', end_char_index=20, file_id=None, start_char_index=0, type='char_location')], text='The grass is green.', type='text'), TextBlock(citations=None, text=' ', type='text'), TextBlock(citations=[CitationCharLocation(cited_text='The sky is blue.', document_index=0, document_title='My Document', end_char_index=36, file_id=None, start_char_index=20, type='char_location')], text='The sky is blue.', type='text')], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=610, output_tokens=45, server_tool_use=None, service_tier='standard'))

In [ ]:
pdf_url = "https://www.princexml.com/samples/invoice-colorful/invoicesample.pdf"
pdf_data = base64.b64encode(httpx.get(pdf_url).content).decode("utf-8")



# pdf citation 
client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": pdf_data
                    },
                    "title": "My Document",
                    "context": "This is a trustworthy document.",
                    "citations": {"enabled": True}
                },
                {
                    "type": "text",
                    "text": "How much are Mangoes?"v
                }
            ]
        }
    ]
)

Message(id='msg_01WAmKLThC3j7RjAXSaxfjyS', content=[TextBlock(citations=[CitationPageLocation(cited_text='Denny Gunawan\r\n221 Queen St\r\nMelbourne VIC 3000\r\n123 Somewhere St, Melbourne VIC 3000 $39.60\r\n(03) 1234 5678\r\nInvoice Number: #20130304\r\nOrganic Items Price/kg Quantity(kg) Subtotal\r\nApple $5.00 1 $5.00\r\nOrange $1.99 2 $3.98\r\nWatermelon $1.69 3 $5.07\r\nMango $9.56 2 $19.12\r\nPeach $2.99 1 $2.99\r\nSubtotal $36.00\r\nGST (10%) $3.60\r\n* Lorem ipsum dolor sit amet, consectetur adipiscing elit. ', document_index=0, document_title='My Document', end_page_number=2, file_id=None, start_page_number=1, type='page_location')], text='Mangoes are priced at $9.56 per kg', type='text'), TextBlock(citations=None, text='.', type='text')], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens

.## Inputs for text/plan dont render well

- Warning when showing the document": Content with content type text/plain must be either start with https:// or be a base64 encoded string.
- Should show contents of the document in plan text
- Not a common use case though. PDFs render well 



https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=34d67b43-36ec-49be-8869-2f3513540044&peeked_trace=34d67b43-36ec-49be-8869-2f3513540044

Anthropic docs: https://docs.claude.com/en/docs/build-with-claude/citations
